## <div style="text-align: center;">Real-time Bitcoin Analysis with PySparkling</div>


In [1]:
pip install h2o_pysparkling_2.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.2/234.2 MB 11.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for h2o_pysparkling_2.3: filename=h2o_pysparkling_2_3-3.46.0.6.post1-py3-none-any.whl size=234470945 sha256=75c7664408d6762fd63b6701e95b9375ebbae95cada7c179b78bb3a4751dee58
  Stored in directory: /root/.cache/pip/wheels/86/ac/0f/599978c498a947287c9dae3c333d57cae03da2f4b6927e4084
Successfully built h2o_pysparkling_2.3
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyspark h2o pysparkling requests pandas matplotlib seaborn findspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 25.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 MB 34.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 47.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 49.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 49.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 47.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.5-py2.py3-none-any.whl size=317747881 sha256=79fdfd1a23bac918f1d796ba339511cc351684d08e51e7ebe5a72875757c54ba
  Stored in directory: /root/.cache/pip/wheels/9e/5b/b4/a3ac8d456edf8c52eb15f9eb3

In [9]:
import os
import findspark

# Set the JAVA_HOME explicitly
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-arm64'
os.environ['SPARK_HOME'] = '/opt/spark-3.5.5-bin-hadoop3'  # Update based on where Spark is installed
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.driver.extraJavaOptions=-Duser.library.path=$SPARK_HOME/lib --conf spark.executor.extraJavaOptions=-Duser.library.path=$SPARK_HOME/lib pyspark-shell'

# Initialize findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.master("local[*]").appName("BitcoinAnalysis").getOrCreate()

# Check the Spark version
print(spark.version)


FileNotFoundError: [Errno 2] No such file or directory: '/opt/spark-3.5.5-bin-hadoop3/./bin/spark-submit'

In [5]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.master("local[*]").appName("BitcoinAnalysis").getOrCreate()

# Check the Spark version
print(spark.version)



JAVA_HOME is not set


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:
pip install requests pandas


In [ ]:
import requests
import pandas as pd

# Function to fetch Bitcoin price from CoinGecko
def fetch_bitcoin_price():
    url = 'https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd'
    response = requests.get(url)
    data = response.json()
    return data['bitcoin']['usd']

# Fetch the latest Bitcoin price
bitcoin_price = fetch_bitcoin_price()
print(f"Bitcoin price in USD: {bitcoin_price}")


In [ ]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

# Initialize Spark session
spark = SparkSession.builder.master("local[*]").appName("BitcoinAnalysis").getOrCreate()

# Create a streaming context with a batch interval of 10 seconds
ssc = StreamingContext(spark.sparkContext, 10)  # 10-second batch interval



In [ ]:
from pyspark.sql.types import StructType, StructField, LongType, DoubleType

# Define the schema for the Bitcoin price data
schema = StructType([
    StructField("timestamp", LongType(), True),
    StructField("price", DoubleType(), True)
])


In [ ]:
import time

# Initialize Spark session
spark = SparkSession.builder.master("local[*]").appName("BitcoinAnalysis").getOrCreate()

# Define the schema for the Bitcoin price data
schema = StructType([
    StructField("timestamp", LongType(), True),
    StructField("price", DoubleType(), True)
])

# Create a streaming DataFrame using the correct path to the directory
bitcoin_stream = spark.readStream.schema(schema).csv("/Users/saanika/Saanika Patil/DATA605/bitcoin_data")

# Perform any data processing here (e.g., cleaning, transformations, etc.)
bitcoin_stream = bitcoin_stream.withColumn("price", bitcoin_stream["price"])

# Define the query that will continuously process the data
query = bitcoin_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Wait for the termination of the stream for a specific time
time.sleep(60)  # Run the stream for 60 seconds
query.stop()  # Stop the stream after 60 seconds


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, DoubleType

# Define the process_batch function to handle each batch of data
def process_batch(df, epoch_id):
    # Process each batch here
    df.show()  # Just display the batch data, you can replace this with your processing logic
    # You can also perform additional transformations, aggregations, or save the data

# Initialize Spark session
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("BitcoinAnalysis") \
    .config("spark.sql.shuffle.partitions", "2") \
    .config("spark.streaming.stopGracefullyOnShutdown", "true") \
    .getOrCreate()

# Define the schema for the Bitcoin price data
schema = StructType([
    StructField("timestamp", LongType(), True),
    StructField("price", DoubleType(), True)
])

# Create a streaming DataFrame using the correct path to the directory
bitcoin_stream = spark.readStream \
    .schema(schema) \
    .csv("/Users/saanika/Saanika Patil/DATA605/bitcoin_data")

# Perform any data processing here (e.g., cleaning, transformations, etc.)
bitcoin_stream = bitcoin_stream.withColumn("price", bitcoin_stream["price"])

# Define the query with foreachBatch processing
query = bitcoin_stream.writeStream \
    .foreachBatch(process_batch) \
    .outputMode("append") \
    .option("checkpointLocation", "/Users/saanika/Saanika Patil/DATA605/bitcoin_data/checkpoint") \
    .start()

# Wait for the termination of the stream
query.awaitTermination()


In [ ]:
# Create a streaming DataFrame with a limit on files per trigger
bitcoin_stream = spark.readStream.schema(schema).option("maxFilesPerTrigger", 1).csv("/Users/saanika/Saanika Patil/DATA605/bitcoin_data")

# Define the query
query = bitcoin_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()


In [ ]:
from h2o.estimators import H2OGradientBoostingEstimator
import h2o

h2o.init()

# Convert pandas DataFrame to H2O frame
train_data = h2o.H2OFrame(pandas_df)

# Define the model
model = H2OGradientBoostingEstimator()
model.train(x=["timestamp"], y="price", training_frame=train_data)

# Evaluate the model
performance = model.model_performance(train_data)
print(performance)


In [ ]:
import matplotlib.pyplot as plt

# Plot the data
def plot_data(data):
    plt.plot(data['timestamp'], data['price'])
    plt.xlabel("Time")
    plt.ylabel("Price (USD)")
    plt.title("Real-time Bitcoin Price")
    plt.show()
